In [35]:
import pandas as pd
import sys, os
import numpy as np
import datetime
import openpyxl
import matplotlib.pyplot as plt
import gc

In [36]:
from snapy import MinHash, LSH

In [37]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [38]:
import random
import re

### Method
#### Use edit distance or levestein distance or LCS to check whether column names are common
#### Use sampling to check zipcode and address
#### Jaccard Similarity, and using Prefix Indexing? I don't think this is quite neccessary since the string is not long

In [39]:
# input path
input_path = os.path.join(os.getcwd(), "data")
# output path
output_path = os.path.join(os.getcwd(), "out")

In [40]:
df = pd.read_csv(os.path.join(input_path, "yellow_tripdata.csv"))

c:\users\hewei\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-06-01 00:31:23,2020-06-01 00:49:58,1.0,3.6,1.0,N,140,68,1.0,15.5,3.0,0.5,4.00,0.0,0.3,23.30,2.5
1,1.0,2020-06-01 00:42:50,2020-06-01 01:04:33,1.0,5.6,1.0,N,79,226,1.0,19.5,3.0,0.5,2.00,0.0,0.3,25.30,2.5
2,1.0,2020-06-01 00:39:51,2020-06-01 00:49:09,1.0,2.3,1.0,N,238,116,2.0,10.0,0.5,0.5,0.00,0.0,0.3,11.30,0.0
3,1.0,2020-06-01 00:56:13,2020-06-01 01:11:38,1.0,5.3,1.0,N,141,116,2.0,17.5,3.0,0.5,0.00,0.0,0.3,21.30,2.5
4,1.0,2020-06-01 00:16:41,2020-06-01 00:29:30,1.0,4.4,1.0,N,186,75,1.0,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5


In [42]:
df["VendorID"].dtype == object

False

In [44]:
list(df.columns.values)

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [31]:
" oo ".strip()

'oo'

In [ ]:
class Settings:
    def __init__(self, inputPath):
        self.dfCountryNames = pd.read_excel(os.path.join(input_path, "countryNames.xlsx"))
        self.dfStateNames = pd.read_excel(os.path.join(input_path, "stateNames.xlsx"))
        self.dfCityNames = pd.read_excel(os.path.join(input_path, "cityNames.xlsx"))
        self.dfCountyNames = pd.read_excel(os.path.join(input_path, "countyNames.xlsx"))

In [62]:
class SpatialColumnDetection:
    def __init__(self, df, settings):
        self.types = ["longitude", "latitude", "country", "state", "city", "county", "address", "zipcode", "other location attribute", 
         "not spatial attribute"]
        
        self.df = df
        # column names
        self.columnNames = list(df.columns)
        # lower case column names
        self.lcColumnNames = []
        # lower case column name to orginal name map
        self.lowerUpperDict = {}
        self.upperLowerDict = {}
        # each column is what type
        self.colNameType = {}
        self.settings = settings
    
    # change all column names to lower cases
    def changeLowerCase(self, ):
        for colName in self.columnNames:
            lwColName = colName.lower()
            self.lcColumnNames.append(lwColName)
            self.lowerUpperDict[lwColName] = colName
            self.upperLowerDict[colName] = lwColName
            
    def lcs(self, X, Y): 
        # find the length of the strings 
        m = len(X) 
        n = len(Y) 

        # declaring the array for storing the dp values 
        L = [[None]*(n + 1) for i in range(m + 1)] 

        """Following steps build L[m + 1][n + 1] in bottom up fashion 
        Note: L[i][j] contains length of LCS of X[0..i-1] 
        and Y[0..j-1]"""
        for i in range(m + 1): 
            for j in range(n + 1): 
                if i == 0 or j == 0 : 
                    L[i][j] = 0
                elif X[i-1] == Y[j-1]: 
                    L[i][j] = L[i-1][j-1]+1
                else: 
                    L[i][j] = max(L[i-1][j], L[i][j-1]) 
  
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1] 
        return L[m][n] 
    
    def editDistance(self, x, y):
        return len(x) + len(y) - 2 * lcs(x, y)
    
    
    def detect(self,):
        self.changeLowerCase()
        for colName in self.columnNames:
            if self.detectLongitude(colName.lower(), df[column]):
                self.colNameType[colName] = "longitude"
            elif self.detectLatitude(colName.lower(), df[column]):
                self.colNameType[colName] = "latitude"
            elif self.detectCountry(colName.lower()):
                self.colNameType[colName] = "country"
            elif self.detectState(colName.lower()):
                self.colNameType[colName] = "state"
            elif self.detectCity(colName.lower()):
                self.colNameType[colName] = "city"
            elif self.detectAddress(colName.lower()):
                self.colNameType[colName] = "address"
            elif detectZipcode(colName.lower()):
                self.colNameType[colName] = "zipcode"
            elif detectOtherLocationAttribute(colName.lower()):
                self.colNameType[colName] = "other location attribute"
            else:
                self.colNameType[colName] = "not spatial attribute"
        return self.colNameType
            
    def commonDetectMethod(self, colName, names, thredshold):
        for name in names:
            if (colName in name) or (name in colName):
                return True
        # use fuzz ratio, which uses the idea of levestain distance, we can specify elsewhere
        for name in names:
            if fuzz.ratio(name, colName) > thredshold:
                return True
        return False
    
    #TODO: to be finish, colName passed in is lowercase
    def detectLongitude(self, colName, column):
        names = ["longitude", "lon"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        return False
    
    
    #TODO
    def detectLatitude(self, colName, column):
        names = ["latitude", "lat"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        return False
    
    # list all the country out, count how many of the sample column values are in country list
    def detectCountry(self, colName):
        # quite similar word
        if "county" in colName:
            return False
        names = ["country"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            dfCountryNames = self.settings.dfCountryNames
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            
            # get the average length of the values
            avgLen = sum( map(len, columnValuesSample) ) / len(columnValuesSample)
            
            # compare with country code, other wise compare with country full name
            if avgLen <= 2.3:
                countries = dfCountryNames["Code"].values
            else:
                countries = dfCountryNames["Name"].values
                
            # equality count
            count = 0
            for value in columnValuesSample:
                for country in countries:
                    if value == country:
                        count += 1
                        break
            if count / sampleLength > 0.6:
                return True
            
        return False
    
    
    # list all the state out, fullname and abbreviation, can use sampling
    def detectState(self, colName):
        names = ["state"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            dfStateNames = self.settings.dfStateNames
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            
            # get the average length of the values
            avgLen = sum( map(len, columnValuesSample) ) / len(columnValuesSample)
            
            # compare with state code, other wise compare with state full name
            if avgLen <= 2.3:
                states = dfStateNames["Code"].values
            else:
                states = dfStateNames["Name"].values
                
            # equality count
            count = 0
            for value in columnValuesSample:
                for state in states:
                    if value == state:
                        count += 1
                        break
            if count / sampleLength > 0.6:
                return True
        return False
    
    # can use sampling
    def detectCity(self, colName):
        names = ["city", "town"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            dfCityNames = self.settings.dfCityNames
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            
            cities = dfCityNames["Name"].values
            # equality count
            count = 0
            for value in columnValuesSample:
                for city in cities:
                    if value == city:
                        count += 1
                        break
            if count / sampleLength > 0.6:
                return True
        return False
    
        # list all the county out, count how many of the sample column values are in county list. Not country!!!
    def detectCounty(self, colName):
        # quite similar word
        if "country" in colName:
            return False
        names = ["county"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            dfCountyNames = self.settings.dfCountyNames
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            
            counties = dfCountyNames["Name"].values
            # equality count
            count = 0
            for value in columnValuesSample:
                for county in counties:
                    if value == county:
                        count += 1
                        break
            if count / sampleLength > 0.6:
                return True
        return False
    
    # need to use sampling
    def detectAddress(self, colName):
        names = ["address"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            
            # get the average length of the values
            avgLen = sum( map(len, columnValuesSample) ) / len(columnValuesSample)
            # probably not address
            if avgLen < 5:
                return False
            
            # use regex expression
            regexPattern = """
            \b\d{1,6} +.{2,25}\b(avenue|ave|court|ct|street|st|drive|dr|lane|ln|road|rd|blvd|plaza|parkway|pkwy|boulevard|)[.,]?(.{0,25} +\b\d{5}\b)?
            """
            count = 0
            for x in columnValuesSample:
                result = re.match(regexPattern, x)
                if result != None:
                    if result.group() > 10:
                        count += 1
            if count / sampleLength > 0.6:
                return True
            
            #TODO use the addr_detection package
#             try:
#                 clf2 = Postal_clf()
#                 result = clf2.predict(columnValuesSample)
#             except:
#                 return False
        return False
    
    # need to use sampling, and regex
    def detectZipcode(self, colName):
        names = ["zip", "zipcode", "code", "zcode", "postcode"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True
        if self.df[colName].dtype == object:
            # sampling and pair wise comparison
            sampleSize = 500
            columnValuesSample = random.sample(list(df[colName].values), sampleSize)
            columnValuesSample = [x for x in columnValuesSample if x == str]
            sampleLength = len(columnValuesSample)
            # meanning many are nan
            if sampleLength / sampleSize < 0.1:
                return False
            # just us country here, can use other api?
            regexPattern = r"^[0-9]{5}(?:-[0-9]{6})?$"
            matches = []
            for x in columnValuesSample:
                result = re.match(regexPattern, x)
                if result != None:
                    matches.append(bool(x))
            count = sum([bool(x) for x in matches])
            if count / sampleLength > 0.6:
                return True
        return False
    
    def detectOtherLocationAttribute(self, colName):
        names = ["location"]
        thredshold = 70
        if self.commonDetectMethod(colName, names, thredshold):
            return True   
        return False
    

In [46]:
"safA_".lower()

'safa_'

In [45]:
random.sample([1,2], 3)

ValueError: Sample larger than population or is negative

In [58]:
fuzz.ratio("longitude1", "longitude")

95

In [57]:
fuzz.partial_ratio("country", "country")

100